In [25]:
from sklearn import svm as sv
import pandas as pd
import pickle
import chess
import numpy as np
#from __future__ import with_statement
import datetime
import time
import os
class svm():
    
    def __init__(self, filename = None, historic = False, gam = 1/69, eval = False):
        if eval is True:
            #gam = 1/64
            self._model = sv.SVR(kernel = 'rbf')
            
        elif filename is None:
                self._model = sv.SVC(kernel = 'rbf', gamma=gam, probability = True)


        elif filename is not None and historic == True:
            #self._model = svm.SVR(kernel = 'rbf')
            try:
                with open(filename, 'rb') as file:
                    self._model = pickle.load(file)
            except EnvironmentError as e:
                print(e)
                
                
        # placeholder!!!
        else: 
            self._model = sv.SVC(kernel = 'rbf', gamma=gam)
      
    
    
    def fit(self, dataset, formatted = True):
        dataset = dataset.drop_duplicates()
        y = dataset.pop('class')
        self._model.fit(dataset,y)
    
    def predict(self, dataset):
        try:
            return self._model.predict(dataset)
        except Exception as e:
            print(e)
            return False


    def predict_proba(self, dataset, formatted = True):
        try:
             return float(self._model.predict_proba(dataset)[:, np.where(self._model.classes_ == 1)].item(0))
        except Exception as e:
            return 1
    
    def write_file(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump(self._model, file)
    
    def load_file(self, filename):
        with open(filename, 'rb') as file:
            self._model = pickle.load(file)


   





class preprocessor(object):

    def __init__(self):
        super().__init__

    def fit(self, filename = None, raw_data = None, dataframe = None):
        if filename is not None:
            self.raw_data = pd.read_csv(filename)
        elif raw_data is not None:
            self.raw_data = pd.DataFrame(raw_data)
        else:
            self.raw_data = dataframe
        return self.raw_data

    def transform(self, predict = False, eval = False):
        data = self.raw_data
        if not predict:
            class_ = data.pop('class')
        data['state'] = data['state'].apply(self.serialize2)
        if not eval:
            data = pd.concat([data['state'].apply(pd.Series), data['move']], axis = 1)
            move = data.pop('move')
            move = move.apply(self.listify).apply(pd.Series)

        #data = pd.concat([data,move.apply(self.listify).apply(pd.Series)], axis = 1)
            move = pd.DataFrame(move).rename(columns = {0:'m1',1:'m2',2:'m3',3:'m4',4:'m5'})

            try:
                move['m5'] = move['m5'].apply(self.encode_move)
            except Exception:
                move['m5'] = 0
        
            move['m1'] = move['m1'].apply(self.encode_move)
            move['m2'] = move['m2'].apply(self.encode_move)
            move['m3'] = move['m3'].apply(self.encode_move)
            move['m4'] = move['m4'].apply(self.encode_move)

        
            data = pd.concat([data,move], axis = 1)
            
            if not predict:
                 data = pd.concat([data,class_], axis = 1)
        else:
            if not predict:
                data = pd.concat([data['state'].apply(pd.Series), class_], axis = 1)
            else:
                data = data['state'].apply(pd.Series)
        return data
    
    def encode_move(self,m):
        try:
            val = ord(m)
            return val
        except Exception:
            return 0



    def listify(self,string):
        return list(str(string))
        
    def bin_matrix_to_decimal(self, state):
        new_state = list()
        ret_state = list()
        for i in range(0,8):
            for j in range(0,8):
                new_state.append((np.array([state[0][i][j],state[1][i][j],state[2][i][j],state[3][i][j]])))
        for i in range(64):
            ret_state.append(int(np.packbits(new_state[i])))
        return ret_state
            

    def serialize(self,board):
        import numpy as np
        assert board.is_valid()

        bstate = np.zeros(64, np.uint8)
        for i in range(64):
            pp = board.piece_at(i)
            if pp is not None:
                #print(i, pp.symbol())
                bstate[i] = {"P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6, \
                        "p": 9, "n":10, "b":11, "r":12, "q":13, "k": 14}[pp.symbol()]
        if board.has_queenside_castling_rights(chess.WHITE):
            assert bstate[0] == 4
            bstate[0] = 7
        if board.has_kingside_castling_rights(chess.WHITE):
            assert bstate[7] == 4
            bstate[7] = 7
        if board.has_queenside_castling_rights(chess.BLACK):
            assert bstate[56] == 8+4
            bstate[56] = 8+7
        if board.has_kingside_castling_rights(chess.BLACK):
            assert bstate[63] == 8+4
            bstate[63] = 8+7

        if board.ep_square is not None:
            assert bstate[board.ep_square] == 0
            bstate[board.ep_square] = 8
        bstate = bstate.reshape(8,8)

        state = np.zeros((4,8,8), np.uint8)
        
        state[0] = (bstate>>3)&1
        state[1] = (bstate>>2)&1
        state[2] = (bstate>>1)&1
        state[3] = (bstate>>0)&1
        """
        state.append(self.bin_matrix_to_decimal((bstate>>3)&1))
        state.append(self.bin_matrix_to_decimal((bstate>>2)&1))
        state.append(self.bin_matrix_to_decimal((bstate>>1)&1))
        state.append(self.bin_matrix_to_decimal((bstate>>0)&1))
        """

        new_state = self.bin_matrix_to_decimal(state)
        new_state.append(board.turn*1.0)
    
        return new_state

    def serialize2(self,board):
        return self.serialize(chess.Board(str(board)))





    
    
        


In [16]:
import json
from pathlib import Path 
import os
ROOT_DIR = Path(os.path.dirname(os.path.realpath('__file__'))).parent
MODELS_DIR = Path(ROOT_DIR / "src" / "models")
HISTORY_DIR = Path(MODELS_DIR / "history")
with open(Path(HISTORY_DIR / 'updated_utility.json'), 'r') as f:
        utils = json.load(f)

In [17]:
new_utils = dict()
c = 0
for row in utils:
    new_utils[c] = [row, utils[row]]
    c+=1
df =pd.DataFrame.from_dict(new_utils,orient='index',columns=['state', 'class'])
df.head()

,state,class
0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,0.0
1,rnbqkbnr/pppp1ppp/8/4p3/8/5N2/PPPPPPPP/RNBQKB1...,1.0
2,rnbqkbnr/ppp2ppp/8/3pN3/8/8/PPPPPPPP/RNBQKB1R ...,10.0
3,rnbqkb1r/ppp1nppp/8/3pN3/8/2N5/PPPPPPPP/R1BQKB...,10.5
4,rnbqkbr1/ppp1nppp/8/3pN3/8/2N1P3/PPPP1PPP/R1BQ...,12.5


In [18]:
model = svm()
prec = preprocessor()
prec.fit(dataframe = df)
data = prec.transform(eval = True)
#data['state'] = data['state'].apply(pd.Series)
data.head()
#model.fit(data)
#model.write_file(Path(HISTORY_DIR / 'eval_model1.pkl'))


,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,class
0,112.0,32.0,48.0,80.0,96.0,48.0,32.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,160.0,240.0,1.0,0.0
1,112.0,32.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,160.0,240.0,1.0,1.0
2,112.0,32.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,160.0,240.0,1.0,10.0
3,112.0,0.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,0.0,240.0,1.0,10.5
4,112.0,0.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,192.0,0.0,1.0,12.5


In [26]:
model = svm(eval = True)
model.fit(data)

In [34]:
def predict_eval(board):
        data=[{'state': board.fen()}]
        prec = preprocessor()
        prec.fit(raw_data = data)
        data = prec.transform(predict = True, eval = True)
        return model.predict(data)


In [35]:
import chess
print(predict_eval(chess.Board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')))

[-0.34784504]


In [36]:
model.write_file(Path(HISTORY_DIR / 'eval_model1.pkl'))

In [ ]:

def predict_probability(board, move):
        data = [{'state':board.fen(),'move':move}]
        #print('predicting for move: ', move)
        prec = preprocessor()
        prec.fit(raw_data = data)
        data = prec.transform(predict = True)
        return self.model.predict_proba(data)

In [ ]:
print(predict_probability(chess.Board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')))

In [37]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,class
0,112.0,32.0,48.0,80.0,96.0,48.0,32.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,160.0,240.0,1.0,0.0
1,112.0,32.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,160.0,240.0,1.0,1.0
2,112.0,32.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,160.0,240.0,1.0,10.0
3,112.0,0.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,0.0,240.0,1.0,10.5
4,112.0,0.0,48.0,80.0,96.0,48.0,0.0,112.0,16.0,16.0,...,240.0,160.0,176.0,208.0,224.0,176.0,192.0,0.0,1.0,12.5


In [38]:
data.to_csv('../src/models/history/raw_data.csv')